In [15]:
import re
import scipy
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.cross_validation import train_test_split
%matplotlib inline
gnb = GaussianNB()

plotly, boca, google maps

In [74]:
sample = pd.read_csv('SF_crime/sampleSubmission.csv')
test = pd.read_csv('SF_crime/test.csv')
test = test.rename(columns={'X': 'Longitude', "Y": "Latitude"})
crime_in_sf = pd.read_csv('SF_crime/train.csv')
crime_in_sf.Dates = pd.to_datetime(crime_in_sf.Dates)
crime_in_sf = crime_in_sf.rename(columns={'X': 'Longitude', "Y": "Latitude",})
crime_in_sf = crime_in_sf.drop(['Resolution', 'Descript'], axis=1)
categories = pd.get_dummies(crime_in_sf['Category'])
day_of_week = pd.get_dummies(crime_in_sf['DayOfWeek'])
pd_district = pd.get_dummies(crime_in_sf['PdDistrict'])
predict = categories.join(day_of_week)
predict = predict.join(pd_district)
predict = predict.rename(columns={'LARCENY/THEFT': 'LARCENY_THEFT', 'DRUG/NARCOTIC': 'DRUG_NARCOTIC', 'FORGERY/COUNTERFEITING': 'FORGERY_COUNTERFEITING', 'PORNOGRAPHY/OBSCENE MAT': 'PORNOGRAPHY_OBSCENE MAT'})
crime_train, crime_test = train_test_split(predict, test_size=.4)

In [120]:
categories = crime_train[['ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY',
       'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE', 'DRUG_NARCOTIC',
       'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION', 'FAMILY OFFENSES',
       'FORGERY_COUNTERFEITING', 'FRAUD', 'GAMBLING', 'KIDNAPPING',
       'LARCENY_THEFT', 'LIQUOR LAWS', 'LOITERING', 'MISSING PERSON',
       'NON-CRIMINAL', 'OTHER OFFENSES', 'PORNOGRAPHY_OBSCENE MAT',
       'PROSTITUTION', 'RECOVERED VEHICLE', 'ROBBERY', 'RUNAWAY',
       'SECONDARY CODES', 'SEX OFFENSES FORCIBLE', 'SEX OFFENSES NON FORCIBLE',
       'STOLEN PROPERTY', 'SUICIDE', 'SUSPICIOUS OCC', 'TREA', 'TRESPASS',
       'VANDALISM', 'VEHICLE THEFT', 'WARRANTS', 'WEAPON LAWS']]
crime_train = crime_train.drop(['ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY',
       'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE', 'DRUG_NARCOTIC',
       'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION', 'FAMILY OFFENSES',
       'FORGERY_COUNTERFEITING', 'FRAUD', 'GAMBLING', 'KIDNAPPING',
       'LARCENY_THEFT', 'LIQUOR LAWS', 'LOITERING', 'MISSING PERSON',
       'NON-CRIMINAL', 'OTHER OFFENSES', 'PORNOGRAPHY_OBSCENE MAT',
       'PROSTITUTION', 'RECOVERED VEHICLE', 'ROBBERY', 'RUNAWAY',
       'SECONDARY CODES', 'SEX OFFENSES FORCIBLE', 'SEX OFFENSES NON FORCIBLE',
       'STOLEN PROPERTY', 'SUICIDE', 'SUSPICIOUS OCC', 'TREA', 'TRESPASS',
       'VANDALISM', 'VEHICLE THEFT', 'WARRANTS', 'WEAPON LAWS'], axis=1)

In [25]:
len(crime_train.Longitude.unique())

32126

In [91]:
crime_train.head()

,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG_NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
128254,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
46702,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
318466,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
450052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
419704,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [3]:
crime_train.head()

,Dates,Category,DayOfWeek,PdDistrict,Address,Longitude,Latitude
525051,2007-12-13 18:15:00,WARRANTS,Thursday,SOUTHERN,STOCKTON ST / MARKET ST,-122.405832,37.785745
319070,2010-12-17 08:00:00,SUSPICIOUS OCC,Friday,INGLESIDE,400 Block of EDINBURGH ST,-122.431371,37.721968
84435,2014-04-01 15:28:00,SUSPICIOUS OCC,Tuesday,INGLESIDE,0 Block of SANTOS ST,-122.418474,37.712104
158461,2013-04-06 15:30:00,SEX OFFENSES FORCIBLE,Saturday,BAYVIEW,1000 Block of REVERE AV,-122.379862,37.725889
475506,2008-09-03 09:30:00,BURGLARY,Wednesday,PARK,600 Block of HAIGHT ST,-122.432940,37.771845


In [50]:
predict.head()

,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG_NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [126]:
transposed_index = np.asarray(crime_train).reshape((-1, 1))
gnb.fit(transposed_index,
        np.asarray(crime_train).reshape((-1, 1)))

/Users/MatthewBarnette/final_project_2/.direnv/python-3.5.0/lib/python3.5/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [128]:
gnb.score(transposed_index, crime_train)

ValueError: Found arrays with inconsistent numbers of samples: [ 526829 8956093]

In [129]:
transposed_index2 = np.asarray(crime_test).reshape((-1, 1))

In [130]:
predictions = gnb.predict(transposed_index2)

In [82]:
test = crime_test[['ARSON', 'Monday']]

In [83]:
test['Predictions'] = predictions

/Users/MatthewBarnette/final_project_2/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [84]:
test.head()

,ARSON,Monday,Predictions
260284,0,0,0
559910,0,0,0
523614,0,0,0
695806,0,0,0
2686,0,0,0


In [86]:
test.Predictions.sum()

644.0

In [66]:
test['Error'] = test.Predictions - test.Monday

/Users/MatthewBarnette/final_project_2/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [72]:
test.Monday.sum() + test.Error.sum()

591.0

In [70]:
test.Error.sum()

-48256.0

In [13]:
categories.head()

,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
525051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
319070,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
84435,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
158461,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
475506,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
day_of_week.head()

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
525051,0,0,0,0,1,0,0
319070,1,0,0,0,0,0,0
84435,0,0,0,0,0,1,0
158461,0,0,1,0,0,0,0
475506,0,0,0,0,0,0,1


In [131]:
y

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  1.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  1.,  0., ...,  0.,  0.,  0.]])

In [124]:
X

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [94]:
SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [147]:
y_pred = gnb.fit(X, y).predict(y)

ValueError: bad input shape (526829, 7)

In [78]:
X = [[0., 0.], [1., 1.]]
y = [0, 1]
SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)
clf = SGDClassifier(loss="hinge", penalty="l2")
clf.fit(X, y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [76]:
clf.predict([[2., 2.]])

array([1])

In [77]:
clf.coef_

array([[ 9.91080278,  9.91080278]])